Use this file as a template to copy and build notebooks

In [ ]:
#@title Mount drive and load libraries
import os
from google.colab import drive

drive.mount('/content/drive/')

!pip install umap-learn[plot]
!wget https://repo.anaconda.com/miniconda/Miniconda3-py39_23.3.1-0-Linux-x86_64.sh
!chmod +x Miniconda3-py39_23.3.1-0-Linux-x86_64.sh
!bash ./Miniconda3-py39_23.3.1-0-Linux-x86_64.sh -b -f -p /usr/local
!conda install -c conda-forge -c bioconda mmseqs2

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix
from scipy.optimize import linear_sum_assignment
import umap

path = '/content/drive/MyDrive/msc_project'
os.chdir(path)

In [ ]:
def cluster_purity(y_true, y_pred):
    confomat = confusion_matrix(y_true, y_pred)
    # We use the Linear Assignment Problem approach to solve label switching problem.
    row_ind, col_ind = linear_sum_assignment(-confomat)
    return confomat[row_ind, col_ind].sum() / np.sum(confomat)

In [ ]:
tensors = pd.read_csv('/content/drive/MyDrive/msc-project-mbalmf01/all_paired/230722_cluster_tensor_scfv_10000.csv', low_memory=False, index_col=0)

Use UMAP to assess effect of increasing components and increasing the number of clusters

In [ ]:
umap_embedding = umap.UMAP(n_components=10, random_state=42)
redux = umap_embedding.fit_transform(tensors.iloc[:, 2:])

mmcluster = tensors['cluster']

In [ ]:
n_cluster = len(set(mmcluster))

In [ ]:
components = [2,5,10,20,30,40,50]
cluster_purities = []
for component in components:
  umap_embedding = umap.UMAP(n_components=component, random_state=42)
  redux = umap_embedding.fit_transform(tensors.iloc[:, 2:])
  kmeans = KMeans(n_clusters=n_cluster, random_state=42, n_init=10)
  kmeans.fit(redux)
  labels = kmeans.labels_
  purity = cluster_purity(y_true=mmcluster, y_pred=labels)
  cluster_purities.append(purity)

print(cluster_purities)

In [ ]:
n_clusters = [20,50,100,200,300,400,500]
cluster_purities = []

umap_embedding = umap.UMAP(n_components=10, random_state=42)
redux = umap_embedding.fit_transform(tensors.iloc[:, 2:])

for cluster in n_clusters:
  kmeans = KMeans(n_clusters=cluster, random_state=42, n_init=10)
  kmeans.fit(redux)
  labels = kmeans.labels_
  purity = cluster_purity(y_true=mmcluster, y_pred=labels)
  cluster_purities.append(purity)

print(cluster_purities)

[0.2664, 0.2859, 0.2913, 0.2583, 0.2406, 0.2346, 0.231]


In [ ]:
kmeans = KMeans(n_clusters=int(round(n_cluster/1.5, 0)), random_state=42)
kmeans.fit(redux)
labels = kmeans.labels_
confusion_matrix_ = confusion_matrix(mmcluster, labels)

row_ind, col_ind = linear_sum_assignment(-confusion_matrix_)
purity = confusion_matrix_[row_ind, col_ind].sum() / np.sum(confusion_matrix_)

print(purity)

0.2264


Assess kmeans by varying the mmseqs cluster threshold

In [ ]:
# @title Varying mmseqs cluster threshold

%%capture

os.chdir('/tmp')
!mkdir /tmp/new_tmp
!cp /content/drive/MyDrive/msc-project-mbalmf01/all_paired/230716_scfv_10000.fasta /tmp
!cp /content/drive/MyDrive/msc-project-mbalmf01/msc-project-source-code-files-22-23-mbalmf01/mmseq_bash.sh /tmp
!chmod +x mmseq_bash.sh
!./mmseq_bash.sh

In [ ]:
date = '230723_'
os.chdir('/content/drive/MyDrive/msc-project-mbalmf01/mmseqs2_output')

dfs = [pd.read_csv(i, index_col=0) for i in os.listdir() if date in i]
new = pd.merge(pd.merge(dfs[0],dfs[1],on='seq'),dfs[2],on='seq')
test = new.merge(right=tensors, how='left', on='seq')

test.head()

,seq,cluster_0.8,cluster_0.9,cluster_0.7,cluster,0,1,2,3,4,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,AGTGGGAGTGACGGTA-1_contig_2_AGTGGGAGTGACGGTA-1...,0,1772,8190,0,0.008540,0.1106,0.016390,0.000094,-0.00847,...,-0.10004,0.015080,-0.07000,-0.07800,-0.024600,-0.015850,0.01553,0.04388,-0.000424,-0.003145
1,CAACCAAAGTAGCCGA-1_contig_1_CAACCAAAGTAGCCGA-1...,2,1789,4460,2,0.005543,0.1104,0.019450,0.000356,-0.01030,...,-0.10400,0.019500,-0.06976,-0.07070,-0.008200,-0.020130,0.00854,0.04288,-0.005417,-0.010160
2,CAACTAGTCCGCAGTG-1_contig_1_CAACTAGTCCGCAGTG-1...,4,5140,7179,4,-0.000502,0.0886,0.003326,0.003420,-0.02446,...,-0.08997,0.010376,-0.06610,-0.07965,-0.012620,-0.004955,0.01124,0.05594,0.005604,-0.001765
3,CATCGAAGTCCAGTGC-1_contig_2_CATCGAAGTCCAGTGC-1...,6,4352,9483,6,-0.002510,0.0939,0.011460,-0.001190,-0.02225,...,-0.08105,0.020650,-0.06950,-0.08640,-0.013756,-0.007060,0.01334,0.04694,0.009895,-0.013310
4,CCGTGGAAGAGGGATA-1_contig_1_CCGTGGAAGAGGGATA-1...,8,1795,463,8,0.020690,0.1050,0.017910,0.011215,-0.01753,...,-0.08746,0.018460,-0.08826,-0.07367,-0.013750,-0.020230,0.03143,0.04926,0.003832,0.004650


Now need to use kmeans to determine if the mmseqs clusters are lining up with the kmeans clusters.